## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lata,PT,40.1629,-8.3327,19.40,85,7,0.97,clear sky
1,1,Sunndalsora,NO,62.6754,8.5515,10.03,80,82,0.08,broken clouds
2,2,Bluff,NZ,-46.6000,168.3333,9.89,90,100,9.78,overcast clouds
3,3,Vaini,TO,-21.2000,-175.2000,21.09,94,90,3.09,moderate rain
4,4,Marsh Harbour,BS,26.5412,-77.0636,26.04,77,66,5.50,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature (celsius) you would like for your trip? "))
max_temp = float(input("What is the maximum temperature (celsius) you would like for your trip? "))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Marsh Harbour,BS,26.5412,-77.0636,26.04,77,66,5.50,broken clouds
10,10,Canton,CN,23.1167,113.2500,28.97,79,100,1.62,light rain
14,14,Hithadhoo,MV,-0.6000,73.0833,28.62,72,15,8.34,few clouds
16,16,Oussouye,SN,12.4850,-16.5469,25.10,77,100,4.33,overcast clouds
18,18,Phuket,TH,8.0000,98.2500,29.49,93,20,3.37,few clouds
20,20,Kashan,IR,33.9831,51.4364,28.51,12,0,1.03,clear sky
21,21,Tiarei,PF,-17.5333,-149.3333,28.89,74,93,7.34,overcast clouds
25,25,Hirara,JP,24.8000,125.2833,26.15,100,75,2.57,broken clouds
35,35,Avarua,CK,-21.2078,-159.7750,26.03,57,20,3.60,few clouds
61,61,Touros,BR,-5.1989,-35.4608,25.50,86,63,4.33,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                158
City                   158
Country                158
Lat                    158
Lng                    158
Max Temp               158
Humidity               158
Cloudiness             158
Wind Speed             158
Current Description    158
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Marsh Harbour,BS,26.04,broken clouds,26.5412,-77.0636,
10,Canton,CN,28.97,light rain,23.1167,113.2500,
14,Hithadhoo,MV,28.62,few clouds,-0.6000,73.0833,
16,Oussouye,SN,25.10,overcast clouds,12.4850,-16.5469,
18,Phuket,TH,29.49,few clouds,8.0000,98.2500,
20,Kashan,IR,28.51,clear sky,33.9831,51.4364,
21,Tiarei,PF,28.89,overcast clouds,-17.5333,-149.3333,
25,Hirara,JP,26.15,broken clouds,24.8000,125.2833,
35,Avarua,CK,26.03,few clouds,-21.2078,-159.7750,
61,Touros,BR,25.50,broken clouds,-5.1989,-35.4608,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><strong>Hotel Name</strong></dt><dd>{Hotel Name}</dd>
<dt><strong>City</strong></dt><dd>{City}</dd>
<dt><strong>Country</strong></dt><dd>{Country}</dd>
<dt><strong>Current Weather</strong></dt><dd>{Current Description} and {Max Temp} °C</dd>
</dl>
"""
# 10a. Get data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))